### Import CICO data from .csv to MongoDB documents (long version)
#### CICO data: https://www.cliclavoro.gov.it/Barometro-Del-Lavoro/Pagine/Microdati-per-la-ricerca.aspx

In [1]:
# Required packages
import pandas as pd
import numpy as np
import json
import os
import pymongo

In [84]:
# Import data (.csv format)
df = pd.read_csv('C:/Users/Nicola Caravaggio/OneDrive/Desktop/Ciu_Tos_Roma3/CICO/data_cico.csv')
print ('Data correctly imported')
# to import all as string, add ", dtype = str" after the path

Data correctly imported


In [85]:
# Convert strings to date format
df['rapporto_datainizio'] =  pd.to_datetime(df['rapporto_datainizio'], format='%d%b%Y')
df['dtfineprevista'] =  pd.to_datetime(df['dtfineprevista'], format='%d%b%Y')
df['dttrasformazione'] =  pd.to_datetime(df['dttrasformazione'], format='%d%b%Y')
df['dtcessazioneeffettiva'] =  pd.to_datetime(df['dtcessazioneeffettiva'], format='%d%b%Y')
df['rapporto_datainizio'] = df['rapporto_datainizio'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['dtfineprevista'] = df['dtfineprevista'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['dttrasformazione'] = df['dttrasformazione'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['dtcessazioneeffettiva'] = df['dtcessazioneeffettiva'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [95]:
# NaN to null
df = df.where((pd.notnull(df)), None)
# Alternatively, Nan to ""
#df = df.replace(np.nan, 'null', regex = True)

In [96]:
# Show first 5 rows of the dataframe
df.head()

,cfdatore_crip,cflavoratore_crip,rapporto_datainizio,annoattivazione,annonascita,codgenere,regione_nascita,codcittadinanza,codtitolostudio,codregionedomicilio,...,codccnl,codagevolazione,idsociolavoratore,dtcessazioneeffettiva,codmotivocessazioneco,dtfineprevista,dttrasformazione,codtipotrasformazione,numeroproroghe,coef_uni
0,48D49BDAC8D8F2B2D63A6122F2B4D332,4C30B254455CDB5FF6E53857746D633E,2017-07-06 00:00:00,2017,1953,Femmina,Campania,Italia,20,Campania,...,235,999,No,2017-07-10 00:00:00,254,2017-07-10 00:00:00,NaT,None,0,7.60417
1,90FF9B696EC2BD0419792F33FA3EBCEE,4CC0E3D4A897A6661D35D4D30D07F57A,2011-12-01 00:00:00,2011,1988,Maschio,Sicilia,Italia,70,Sicilia,...,999,999,No,2012-03-31 00:00:00,254,2012-03-31 00:00:00,NaT,None,0,7.625
2,D23485B84D16E0F9CCDB7B7F013B4F76,57E6FA30170F062738665433915DEAFD,2015-04-22 00:00:00,2015,1960,Femmina,Campania,Italia,10,Campania,...,11,999,No,2015-06-30 00:00:00,254,2015-06-30 00:00:00,NaT,None,0,7.60417
3,182D316EB9A78C40E0CF5A2D5C25056C,49AF87408CC71FBB51C01D6D0C90BC3B,2016-08-01 00:00:00,2016,1990,Maschio,Estero,Extra UE,40,Veneto,...,257,999,No,2018-01-22 00:00:00,10,NaT,NaT,None,0,5.43698
4,90D5C65C1557742DBE2808A10956456D,64D130D252AE4739EC6D7EF9197D76DE,2011-06-01 00:00:00,2011,1970,Maschio,Estero,Extra UE,0,Campania,...,272,999,No,2012-01-31 00:00:00,12,NaT,NaT,None,0,5.45187


In [53]:
# Check columns' types
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
cfdatore_crip                1000 non-null object
cflavoratore_crip            1000 non-null object
rapporto_datainizio          1000 non-null object
annoattivazione              1000 non-null int64
annonascita                  1000 non-null int64
codgenere                    1000 non-null object
regione_nascita              1000 non-null object
codcittadinanza              1000 non-null object
codtitolostudio              1000 non-null int64
codregionedomicilio          1000 non-null object
codprovincialavoro           1000 non-null int64
codregionelavoro             1000 non-null object
codsettore                   1000 non-null object
codtipocontratto             1000 non-null int64
codtipoorario                1000 non-null object
codqualificaprofessionale    1000 non-null object
codccnl                      1000 non-null int64
codagevolazione              1000 non-null int64
i

In [54]:
# Convert int64 (not supported in MongoDB) into float64
df["annoattivazione"] = df["annoattivazione"].astype(np.float64)
df["annonascita"] = df["annonascita"].astype(np.float64)
df["codtitolostudio"] = df["codtitolostudio"].astype(np.float64)
df["codprovincialavoro"] = df["codprovincialavoro"].astype(np.float64)
df["codtipocontratto"] = df["codtipocontratto"].astype(np.float64)
df["codccnl"] = df["codccnl"].astype(np.float64)
df["codagevolazione"] = df["codagevolazione"].astype(np.float64)
df["numeroproroghe"] = df["numeroproroghe"].astype(np.float64)

In [55]:
# Descriptive statistics
df.describe()

,annoattivazione,annonascita,codtitolostudio,codprovincialavoro,codtipocontratto,codccnl,codagevolazione,numeroproroghe
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,2012.252000,1975.45200,26.131000,50.234000,9.02200,175.835000,968.221000,0.162000
std,5.116119,12.30663,21.789599,28.451375,15.35993,172.159422,166.765323,0.538559
min,1974.000000,1942.00000,0.000000,1.000000,1.00000,1.000000,8.000000,0.000000
25%,2010.000000,1967.00000,7.500000,22.000000,2.00000,47.000000,999.000000,0.000000
50%,2013.000000,1977.00000,20.000000,58.000000,2.00000,187.500000,999.000000,0.000000
75%,2016.000000,1985.00000,40.000000,72.000000,3.00000,264.000000,999.000000,0.000000
max,2018.000000,2007.00000,90.000000,110.000000,71.00000,999.000000,999.000000,7.000000


#### Generate a .json-type file from the imported .csv with nested arrays

In [56]:
from datetime import datetime

# Encode variables for .json
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()

        return json.JSONEncoder.default(self, o)

# Create a document for each worker (cflavoratore_crip) with a nested array for each working contract (attivazioni)
finalList = []
finalDict = {}
grouped = df.groupby(['cflavoratore_crip', 
                      'annonascita',
                      'codgenere',
                      'regione_nascita',
                      'codcittadinanza',
                      'codtitolostudio',
                      'coef_uni'
                     ])
for key, value in grouped:

    dictionary = {}

    j = grouped.get_group(key).reset_index(drop=True)
    dictionary['cflavoratore_crip'] = j.at[0, 'cflavoratore_crip']
    dictionary['annonascita'] = j.at[0, 'annonascita']
    dictionary['codgenere'] = j.at[0, 'codgenere']
    dictionary['regione_nascita'] = j.at[0, 'regione_nascita']
    dictionary['codcittadinanza'] = j.at[0, 'codcittadinanza']
    dictionary['codtitolostudio'] = j.at[0, 'codtitolostudio']
    dictionary['coef_uni'] = j.at[0, 'coef_uni']
    
    dictList = []
    anotherDict = {}
    for i in j.index:

        anotherDict['cfdatore_crip'] = j.at[i, 'cfdatore_crip']
        anotherDict['rapporto_datainizio'] = j.at[i, 'rapporto_datainizio']
        anotherDict['codregionedomicilio'] = j.at[i, 'codregionedomicilio']
        anotherDict['codregionelavoro'] = j.at[i, 'codregionelavoro']
        anotherDict['codprovincialavoro'] = j.at[i, 'codprovincialavoro']
        anotherDict['codsettore'] = j.at[i, 'codsettore']
        anotherDict['codtipocontratto'] = j.at[i, 'codtipocontratto']
        anotherDict['codtipoorario'] = j.at[i, 'codregionedomicilio']
        anotherDict['codqualificaprofessionale'] = j.at[i, 'codqualificaprofessionale']
        anotherDict['codccnl'] = j.at[i, 'codccnl']
        anotherDict['codagevolazione'] = j.at[i, 'codagevolazione']
        anotherDict['idsociolavoratore'] = j.at[i, 'idsociolavoratore']
        anotherDict['dtcessazioneeffettiva'] = j.at[i, 'dtcessazioneeffettiva']
        anotherDict['dtfineprevista'] = j.at[i, 'dtfineprevista']
        anotherDict['dttrasformazione'] = j.at[i, 'dttrasformazione']
        anotherDict['codtipotrasformazione'] = j.at[i, 'codtipotrasformazione']
        anotherDict['numeroproroghe'] = j.at[i, 'numeroproroghe']        

        dictList.append(anotherDict.copy())

    dictionary['attivazioni'] = dictList

    finalList.append(dictionary)

# Note: the following commands are useful just to show the structure, 
# the objsect that will be imported into MongoDB is 'finalList'
cico_final = (json.dumps(finalList, indent = 4, cls = NpEncoder))
print(json.dumps(finalList, indent = 4, cls = NpEncoder))

[
    {
        "cflavoratore_crip": "00753DCF12E23D69F5E4CF95A04700AC",
        "annonascita": 1978.0,
        "codgenere": "Maschio",
        "regione_nascita": "Puglia",
        "codcittadinanza": "Italia",
        "codtitolostudio": 40.0,
        "coef_uni": 7.625,
        "attivazioni": [
            {
                "cfdatore_crip": "6C1DFCC6596D219ADAAE5ABA9C853015",
                "rapporto_datainizio": "2009-12-30 00:00:00",
                "codregionedomicilio": "Puglia",
                "codregionelavoro": "Puglia",
                "codprovincialavoro": 73.0,
                "codsettore": "56.10.11",
                "codtipocontratto": 2.0,
                "codtipoorario": "Puglia",
                "codqualificaprofessionale": "5.2.2.1",
                "codccnl": 264.0,
                "codagevolazione": 999.0,
                "idsociolavoratore": "No",
                "dtcessazioneeffettiva": "2010-01-01 00:00:00",
                "dtfineprevista": "2010-01-01 00:00:00",

#### Import data into MongoDB

In [71]:
# Import data into MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["cico"]

print(db.list_collection_names())
print("The collection does exist?")
print('datacico' in db.list_collection_names())    
collection = db['datacico']
collection.estimated_document_count() == 0
collection.drop()

col = db['datacico']

result = col.insert_many(finalList)

# Number of object inserted
inserted = result.inserted_ids
print("Number of inserted objects:") 
len(inserted)

['datacico']
The collection does exist?
False
Number of inserted objects:


958

In [ ]:
# To save as .js format
#with open('C:/Users/Nicola Caravaggio/OneDrive/Desktop/Ciu_Tos_Roma3/CICO/data_cico.js', 'w') as 
#f: json.dump(cico_final, f, indent = 4, cls = NpEncoder)

In [3]:
# Modify strings into Date ISO format (into MongoDB)
client = pymongo.MongoClient('localhost', 27017)
db = client['cico']
datacico = db['datacico']
pipe = [{
        '$addFields': {
            'attivazioni': {
                '$map': {
                    'input': '$attivazioni', 
                    'as': 'att', 
                    'in': {  
                        'cfdatore_crip': '$$att.cfdatore_crip',
                        'rapporto_datainizio': 
                            { '$toDate': { '$substr': [ '$$att.rapporto_datainizio', 0, { '$subtract': [ {
                                '$strLenCP': '$$att.rapporto_datainizio' }, 1 ] } ] } },
                        'codregionedomicilio': '$$att.codregionedomicilio', 
                        'codregionelavoro': '$$att.codregionelavoro',
                        'codprovincialavoro': '$$att.codprovincialavoro', 
                        'codsettore': '$$att.codsettore', 
                        'codtipocontratto': '$$att.codtipocontratto', 
                        'codtipoorario': '$$att.codtipoorario', 
                        'codqualificaprofessionale': '$$att.codqualificaprofessionale', 
                        'codccnl': '$$att.codccnl', 
                        'codagevolazione': '$$att.codagevolazione',  
                        'idsociolavoratore': '$$att.idsociolavoratore',
                        'dtcessazioneeffettiva': 
                            { '$cond': [ { '$eq': [ "$$att.dtcessazioneeffettiva", "NaT" ] }, "NaT", 
                                { '$toDate': { '$substr': [ '$$att.dtcessazioneeffettiva', 0, { '$subtract': [ {
                                    '$strLenCP': '$$att.dtcessazioneeffettiva' }, 1 ] } ] } }
                                 ] },
                        'dtfineprevista': 
                            { '$cond': [ { '$eq': [ "$$att.dtfineprevista", "NaT" ] }, "NaT", 
                                { '$toDate': { '$substr': [ '$$att.dtfineprevista', 0, { '$subtract': [ {
                                    '$strLenCP': '$$att.dtfineprevista' }, 1 ] } ] } }
                                 ] },
                        'dttrasformazione': 
                            { '$cond': [ { '$eq': [ "$$att.dttrasformazione", "NaT" ] }, "NaT", 
                                { '$toDate': { '$substr': [ '$$att.dttrasformazione', 0, { '$subtract': [ {
                                    '$strLenCP': '$$att.dttrasformazione' }, 1 ] } ] } }
                                 ] },
                        'codtipotrasformazione': '$$att.codtipotrasformazione', 
                        'numeroproroghe': '$$att.numeroproroghe'
                    }
                }
            }
        }
    }, {
        '$out': 'datacico'
    }]

TestOutput = datacico.aggregate(pipeline = pipe)
client.close()
print(list(TestOutput))
print("Query correctly read")

[]
Query correctly read
